<a href="https://colab.research.google.com/github/shivangsingh26/FL-BC-BTP/blob/master/FedWPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow federated
!pip install gdown

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Unzip the datasets
zip_path = '/content/drive/My Drive/BTP.zip'
unzip_path = '/content/datasets'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)


Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to load and preprocess data with augmentation
def load_data(client_path):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(client_path, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(client_path, 'test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

clients = ["/content/datasets/Non-iid datasets/non_iid_subset_1","/content/datasets/Non-iid datasets/non_iid_subset_2","/content/datasets/Non-iid datasets/non_iid_subset_3","/content/datasets/Non-iid datasets/non_iid_subset_4",]
data_paths = [os.path.join(unzip_path, client) for client in clients]

train_generators = []
test_generators = []
for path in data_paths:
    train_gen, test_gen = load_data(path)
    train_generators.append(train_gen)
    test_generators.append(test_gen)


Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.


In [3]:
def create_model(num_classes=19):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
import numpy as np

# Define number of communication rounds and hyperparameters grid
num_rounds = 5
hyperparameters_grid = {
    'RR': [0.3, 0.5, 0.7],
    'epochs': [5, 10]
}

def fed_wpr(client_models, RR):
    new_weights_list = []
    client_weights = [model.get_weights() for model in client_models]

    for client_id in range(len(client_models)):
        new_weights = []
        for layer_weights in zip(*client_weights):
            weighted_sum = np.zeros_like(layer_weights[0])
            for j in range(len(client_models)):
                weighted_sum += RR * layer_weights[j]
            personalized_weights = (1 - RR) * layer_weights[client_id] + weighted_sum
            new_weights.append(personalized_weights)
        new_weights_list.append(new_weights)

    return new_weights_list

def train_and_evaluate(client_models, train_generators, test_generators, RR, epochs, num_rounds=10):
    for round_num in range(num_rounds):
        print(f'Round {round_num+1}/{num_rounds}')

        for i in range(4):
            print(f'Training client {i+1}')
            client_models[i].fit(train_generators[i], epochs=epochs, validation_data=test_generators[i])

        new_weights_list = fed_wpr(client_models, RR)

        for i in range(4):
            client_models[i].set_weights(new_weights_list[i])

    avg_accuracy = 0
    for i in range(4):
        loss, accuracy = client_models[i].evaluate(test_generators[i])
        avg_accuracy += accuracy
        print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

    avg_accuracy /= 4
    return avg_accuracy

def grid_search(hyperparameters_grid):
    best_accuracy = 0
    best_params = {}

    for RR in hyperparameters_grid['RR']:
        for epochs in hyperparameters_grid['epochs']:
            print(f'Evaluating RR = {RR}, epochs = {epochs}')
            client_models = [create_model(num_classes=19) for _ in range(4)]

            avg_accuracy = train_and_evaluate(client_models, train_generators, test_generators, RR, epochs)

            print(f'Average accuracy for RR = {RR}, epochs = {epochs}: {avg_accuracy}')

            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = {'RR': RR, 'epochs': epochs}

    return best_params, best_accuracy

best_params, best_accuracy = grid_search(hyperparameters_grid)
print(f'Best parameters: {best_params} with average accuracy: {best_accuracy}')


Evaluating RR = 0.3, epochs = 5
9406464/9406464 [==============================] - 2s 0us/step
Round 1/10
Training client 1
Epoch 1/5
595/595 [==============================] - 322s 526ms/step - loss: 2.5215 - accuracy: 0.2138 - val_loss: 2.2394 - val_accuracy: 0.3317
Epoch 2/5
595/595 [==============================] - 266s 446ms/step - loss: 2.2101 - accuracy: 0.3029 - val_loss: 1.8453 - val_accuracy: 0.4164
Epoch 3/5
595/595 [==============================] - 263s 442ms/step - loss: 2.1064 - accuracy: 0.3348 - val_loss: 1.6908 - val_accuracy: 0.4558
Epoch 4/5
595/595 [==============================] - 262s 440ms/step - loss: 2.0058 - accuracy: 0.3617 - val_loss: 1.5987 - val_accuracy: 0.5065
Epoch 5/5
595/595 [==============================] - 265s 445ms/step - loss: 1.9645 - accuracy: 0.3811 - val_loss: 1.6173 - val_accuracy: 0.4941
Training client 2
Epoch 1/5
600/600 [==============================] - 328s 539ms/step - loss: 2.3289 - accuracy: 0.2792 - val_loss: 1.6640 - val_accur

In [ ]:
for i in range(4):
    loss, accuracy = client_models[i].evaluate(test_generators[i])
    print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')